In [2]:
import pandas as pd

df = pd.read_csv('ligue180-primeiro-semestre-2024.csv', delimiter=';', low_memory=False)

df = df.fillna('NÃO REGISTRADO')

df['Data_de_cadastro'] = pd.to_datetime(df['Data_de_cadastro']).dt.date

df['Município'] = df['Município'].str.extract(r'\| (.*)')

df_filtrado = df[df['UF'].isin(['SP', 'RJ', 'MG', 'BA', 'RS', 'PE', 'PR', 'SC', 'GO', 'CE'])]



frequencia = df_filtrado[
    (df_filtrado['Frequência'] == 'DIARIAMENTE') & 
    ~(df_filtrado['Denúncia_emergencial'] == 'NÃO')
]


frequencia_diaria = frequencia.groupby(['Data_de_cadastro', 'Denunciante', 'Cenário_da_violação', 'Município', 'UF', 'Denúncia_emergencial']).size().reset_index(name='Frequência_Diaria')

media_frequencia = frequencia_diaria.groupby(['Data_de_cadastro', 'Denunciante', 'Cenário_da_violação', 'Município', 'UF', 'Denúncia_emergencial'])['Frequência_Diaria'].mean().reset_index(name='Media_Frequência_Diaria')

media_frequencia.sort_values(by='Media_Frequência_Diaria', ascending=False).to_csv('media_frequencia.csv', index=False)


In [3]:
import boto3

s3 = boto3.client('s3', region_name='us-east-1')


arquivo_local = './media_frequencia.csv'
bucket_name = 'desafio-compass-aws'

s3.upload_file(arquivo_local, bucket_name, 'media_frequencia.csv')
print("Arquivo enviado com sucesso!")

Arquivo enviado com sucesso!
